### Assignment 5

![image info](https://www.mdpi.com/sensors/sensors-17-01951/article_deploy/html/images/sensors-17-01951-g003.png)


>SqueezeNet: AlexNet-level accuracy with 50x fewer parameters and <0.5MB model size
(https://arxiv.org/abs/1602.07360)

In this assigment you're going to use the pretrained network SqueezeNetv1.2 (~ 5 Mb) 

#### Task 1 (5 points):

go to https://github.com/miaow1988/SqueezeNet_v1.2 and download the 'symbol.json' and '.params' files (there is not a 'synset.txt' file! so don't use these lines, Hint: just comment these lines).

* Install MXNet v1.5 (hint: create a new conda environmet with python 3, pip install mxnet==1.5.1) and follow the same steps of the lecture (part: *Using pre-trained models as feature extractors*). Find the flatten output layer and create a feature extractor (hint: It should be a numpy array of 1000 elements).
* Download the dogs versus cats *training folder* from https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition/data (Remember the number of images is 12500 for each class).
* Extract the array of features for different number of images (N: 10, 100, 500, 1000, also 5000 and 12500) and for each value train your favorite binary classifier (only one!!!) using GridSearch to optimize some hyperparameters. Consider to use https://notebooks.csc.fi if you have computational limitations. 

* Report the accuracy for each value of N and the computational time during the training step.

#### Task 2 (5 points):

Repeat all previous steps using MobileNet V2 (https://github.com/KeyKy/mobilenet-mxnet). How the two networks compare in terms of accuracy and running time?

**WARNING**: At least for N= 5000 and 12500 it can take some time in your computer, depending of your resources. The time can largely increases depending of your chosen classifier.

#### Task 3 (5 points):

Using the best network. Train a machine learning model able to predic COVID-19 from chest X-Ray images. Use the data from https://www.kaggle.com/praveengovi/coronahack-chest-xraydataset. 

Present and discuss your best model.


# TASK2:

In [ ]:
#! pip install mxnet-cu101==1.5 --user

In [ ]:
#! pip install mxnet==1.5.1 --user
#! pip install requests matplotlib opencv-python-headless

In [1]:
import mxnet as mx1
path='https://github.com/KeyKy/mobilenet-mxnet/'
[mx1.test_utils.download(path+'raw/master/mobilenet_v2-symbol.json'),
 mx1.test_utils.download(path+'raw/master/mobilenet_v2-0000.params')]
# mx.test_utils.download(path+'synset.txt')

['mobilenet_v2-symbol.json', 'mobilenet_v2-0000.params']

In [14]:
sym1, arg_params1, aux_params1 = mx1.model.load_checkpoint("./mobilenet_v2", 0)

In [15]:
mobile_mod = mx1.mod.Module(symbol=sym1, context=mx1.cpu(), label_names=None)
mobile_mod.bind(for_training=False, data_shapes=[('data', (1,3,224,224))], 
          label_shapes=mobile_mod._label_shapes)
mobile_mod.set_params(arg_params1, aux_params1, allow_missing=True)
#with open('model-symbol.json', 'r') as f:
#    for n in range(10):
#        s = f.readline().strip()
#        if s: print(s)
with open('mobilenet_v2-symbol.json', 'r') as f1:
    labels = [l.rstrip() for l in f1]

In [5]:
#! pip install opendatasets

     |████████████████████████████████| 97 kB 2.7 MB/s eta 0:00:011
     |████████████████████████████████| 58 kB 4.7 MB/s eta 0:00:011
     |████████████████████████████████| 78 kB 4.3 MB/s eta 0:00:011
  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73053 sha256=beb79bfea759d8a194f50282a92b2c2494eee7533c10a38978c0503b8177c4fe
  Stored in directory: /home/jovyan/.cache/pip/wheels/62/d6/58/5853130f941e75b2177d281eb7e44b4a98ed46dd155f556dc5
Successfully built kaggle


In [6]:
#! pip install opendatasets
#import opendatasets as od
#import zipfile as zf
#od.download("https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition/data", force=True)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: abhimolaj
Your Kaggle Key: ········


  0%|          | 1.00M/814M [00:00<02:18, 6.15MB/s]

100%|██████████| 814M/814M [00:20<00:00, 42.6MB/s] 



Extracting archive ./dogs-vs-cats-redux-kernels-edition/dogs-vs-cats-redux-kernels-edition.zip to ./dogs-vs-cats-redux-kernels-edition


In [7]:
#files = zf.ZipFile("dogs-vs-cats-redux-kernels-edition/train.zip", 'r')
#files.extractall('images')
#files.close()

In [16]:
from os import listdir
from os.path import isfile, join
import os

cats_imgs = [join('images/train',f1) for f1 in listdir('images/train') if f1.startswith('cat')]
dogs_imgs = [join('images/train',f1) for f1 in listdir('images/train') if f1.startswith('dog')]

In [17]:
print("cats: {} and dogs: {}".format(len(cats_imgs),len(dogs_imgs)))

cats: 12500 and dogs: 12500


In [18]:
# list the last 10 layers
all_layers = sym1.get_internals()
all_layers.list_outputs()[-10:]


['conv6_4_bn_moving_var',
 'conv6_4_bn_output',
 'relu6_4_output',
 'pool6_output',
 'fc7_weight',
 'fc7_bias',
 'fc7_conv_output',
 'fc7_flatten_output',
 'prob_label',
 'prob_output']

In [19]:
#Define a simple batch
%matplotlib inline
import matplotlib.pyplot as plt
import cv2
import numpy as np
# define a simple data batch
from collections import namedtuple
Batch = namedtuple('Batch', ['data'])

In [28]:
def get_image(url, show=False):
    if url.startswith('http'):
        # download and show the image
        fname = mx1.test_utils.download(url)
    else:
        fname = url
    img = cv2.cvtColor(cv2.imread(fname), cv2.COLOR_BGR2RGB)
    if img is None:
         return None
    if show:
         plt.imshow(img)
         plt.axis('off')
    # convert into format (batch, RGB, width, height)
    img = cv2.resize(img, (224, 224))
    img = np.swapaxes(img, 0, 2)
    img = np.swapaxes(img, 1, 2)
    img = img[np.newaxis, :]
    return img

def predict(url):
    img = get_image(url, show=True)
    # compute the predict probabilities
    mod.forward(Batch([mx1.nd.array(img)]))
    prob = mod.get_outputs()[0].asnumpy()
    # print the top-5
    prob = np.squeeze(prob)
    a = np.argsort(prob)[::-1]
    for i in a[0:5]:
        print('probability=%f, class=%s' %(prob[i], labels[i]))


In [23]:
fe_sym = all_layers['fc7_flatten_output']
fe_mod = mx1.mod.Module(symbol=fe_sym, context=mx1.cpu(), label_names=None)
fe_mod.bind(for_training=False, data_shapes=[('data', (1,3,224,224))])
fe_mod.set_params(arg_params1, aux_params1)

In [29]:
def get_features(img):
    fe_mod.forward(Batch([mx1.nd.array(img)]))
    features = fe_mod.get_outputs()[0].asnumpy()
    return features

In [30]:
Nmax = 10
cats_features = [get_features(get_image(img)).ravel() for img in cats_imgs[:Nmax]]
dogs_features = [get_features(get_image(img)).ravel() for img in dogs_imgs[:Nmax]]

In [31]:
Y_cats = np.array(Nmax * [1])
Y_dogs = np.array(Nmax * [0])

In [32]:
X_cvd = np.vstack([cats_features,dogs_features])
Y_cvd = np.vstack([Y_cats,Y_dogs]).ravel()

In [33]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from time import time
import warnings

param_grid = {'C': [0.001, 0.01, 0.1, 1, 3, 5, 7, 10, 12, 20, 120],
              'gamma': [0.001, 0.01, 0.1, 1, 3, 5, 7, 10, 12, 20, 100]}
grid_search = GridSearchCV(SVC(), param_grid, cv=5)
X_train, X_test, y_train, y_test = train_test_split(X_cvd, Y_cvd, random_state=0)

start = time()
grid_search.fit(X_train, y_train)
#report(grid_search.cv_results_)
stop = time()

In [35]:
print("Test set score: {:.2f}".format(grid_search.score(X_test, y_test)))
print("gridSearchCV took %.2f seconds" % ((stop - start)))
print("Best parameters: {}".format(grid_search.best_params_))
print("Best cross-validation score: {:.2f}\n".format(grid_search.best_score_))
print("Best estimator:\n{}".format(grid_search.best_estimator_))

Test set score: 0.40
gridSearchCV took 1.10 seconds
Best parameters: {'C': 0.001, 'gamma': 0.01}
Best cross-validation score: 0.53

Best estimator:
SVC(C=0.001, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.01, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)


As we see the above results, for Nmax=10, we have used Gridsearch SVC model which took 1.10 seconds time and 
best cross validation score achieved is 0.53 with hyper parameter values when C is 0.001 and gamma is 0.01. test score is 0.40

In [36]:
Nmax100 = 100
cats_features100 = [get_features(get_image(img)).ravel() for img in cats_imgs[:Nmax100]]
dogs_features100 = [get_features(get_image(img)).ravel() for img in dogs_imgs[:Nmax100]]

Y_cats100 = np.array(Nmax100 * [1])
Y_dogs100 = np.array(Nmax100 * [0])

X_cvd100 = np.vstack([cats_features100,dogs_features100])
Y_cvd100 = np.vstack([Y_cats100,Y_dogs100]).ravel()

grid_search = GridSearchCV(SVC(), param_grid, cv=5)
X_train, X_test, y_train, y_test = train_test_split(X_cvd100, Y_cvd100, random_state=0)

start = time()
grid_search.fit(X_train, y_train)
stop = time()

print("Test set score100: {:.2f}".format(grid_search.score(X_test, y_test)))
print("gridSearchCV100 took %.2f seconds" % ((stop - start)))
print("Best parameters100: {}".format(grid_search.best_params_))
print("Best cross-validation score100: {:.2f}\n".format(grid_search.best_score_))
print("Best estimator100:\n{}".format(grid_search.best_estimator_))

Test set score100: 0.46
gridSearchCV100 took 13.40 seconds
Best parameters100: {'C': 0.001, 'gamma': 0.001}
Best cross-validation score100: 0.51

Best estimator100:
SVC(C=0.001, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)


As we see the above results, for Nmax=100, we have used Gridsearch SVC model which took 13.40 seconds time and 
best cross validation score achieved is 0.51 with hyper parameter values when C is 0.001 and gamma is 0.001. test score is 0.46

In [37]:
Nmax500 = 500
cats_features500 = [get_features(get_image(img)).ravel() for img in cats_imgs[:Nmax500]]
dogs_features500 = [get_features(get_image(img)).ravel() for img in dogs_imgs[:Nmax500]]

Y_cats500 = np.array(Nmax500 * [1])
Y_dogs500 = np.array(Nmax500 * [0])

X_cvd500 = np.vstack([cats_features500,dogs_features500])
Y_cvd500 = np.vstack([Y_cats500,Y_dogs500]).ravel()

grid_search = GridSearchCV(SVC(), param_grid, cv=5)
X_train, X_test, y_train, y_test = train_test_split(X_cvd500, Y_cvd500, random_state=0)

start = time()
grid_search.fit(X_train, y_train)
stop = time()

print("Test set score500: {:.2f}".format(grid_search.score(X_test, y_test)))
print("gridSearchCV500 took %.2f seconds" % ((stop - start)))
print("Best parameters500: {}".format(grid_search.best_params_))
print("Best cross-validation score500: {:.2f}\n".format(grid_search.best_score_))
print("Best estimator500:\n{}".format(grid_search.best_estimator_))

Test set score500: 0.48
gridSearchCV500 took 313.06 seconds
Best parameters500: {'C': 0.001, 'gamma': 0.001}
Best cross-validation score500: 0.51

Best estimator500:
SVC(C=0.001, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)


As we see the above results, for Nmax=500, we have used Gridsearch SVC model which took 313.06 seconds time and 
best cross validation score achieved is 0.51 with hyper parameter values when C is 0.001 and gamma is 0.001. test score is 0.48

In [38]:
Nmax1000 = 1000
cats_features1000 = [get_features(get_image(img)).ravel() for img in cats_imgs[:Nmax1000]]
dogs_features1000 = [get_features(get_image(img)).ravel() for img in dogs_imgs[:Nmax1000]]

Y_cats1000 = np.array(Nmax1000 * [1])
Y_dogs1000 = np.array(Nmax1000 * [0])

X_cvd1000 = np.vstack([cats_features1000,dogs_features1000])
Y_cvd1000 = np.vstack([Y_cats1000,Y_dogs1000]).ravel()

grid_search = GridSearchCV(SVC(), param_grid, cv=5)
X_train, X_test, y_train, y_test = train_test_split(X_cvd1000, Y_cvd1000, random_state=0)

start = time()
grid_search.fit(X_train, y_train)
stop = time()

print("Test set score1000: {:.2f}".format(grid_search.score(X_test, y_test)))
print("gridSearchCV1000 took %.2f seconds" % ((stop - start)))
print("Best parameters1000: {}".format(grid_search.best_params_))
print("Best cross-validation score1000: {:.2f}\n".format(grid_search.best_score_))
print("Best estimator1000:\n{}".format(grid_search.best_estimator_))

Test set score1000: 0.49
gridSearchCV1000 took 1233.53 seconds
Best parameters1000: {'C': 0.001, 'gamma': 0.001}
Best cross-validation score1000: 0.50

Best estimator1000:
SVC(C=0.001, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)


As we see the above results, for Nmax=1000, we have used Gridsearch SVC model which took 1233.53 seconds time and 
best cross validation score achieved is 0.50 with hyper parameter values when C is 0.001 and gamma is 0.001. test score is 0.49

As we see, the improved scores for Mobilenet are achieved when Nmax=1000 as compared to previous values of Nmax.

===========================================================================================================
###However, when we compare performance of Mobilenet model scores with SqueezeNet w.r.t. time and score, for the dogs vs cat same set of images. 
We can see there is major difference in scores and time. 
Squeezenet provides far better scores and processesing time is far less than mobilnet for same Nmax values. 

Hence, we conclude that SqueeNet is better suitable model for this data as compared to Mobilenet model.
================================================================

Please see: due to performance issue, the Nmax values 5000 and 12500 respectively are not used and scores are not calculated.
